# Decision Process of Network Hyper-Parameters
## Prepared by Furkan Küçük for DataBoss Analytics Job Application

Decision process of a machine learning pipeline can get very trivial. In this notebook, I will try to explain my general approach on hyper-parameter tuning. 

First of all, hyper-parameter decision process might take some time. This may be due to shortage of resources (like having a low-spec hardware) or time. Especially tasks on computer vision may take some time since datasets of computer vision tasks may be challenging, have clutter, hard to label etc. Besides, since computer vision tasks are relatively more complicated, it may need more complex deep learning architectures.

Important note: This task is being done with a relatively low-spec hardware(Google Colab). Hence, one may need to tune hyper-parameters with pre-acquired insights. The experiments will be held with a light network architecture and will have an assumption for found parameters will apply for all conditions. However, despite this is not an accurate assumption since different hyper-parameters may perform better under different conditions, this experiments can provide a good start point for further hyper-parameter tuning.

In [1]:
# imports
import math
import os
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn.functional as F
from dataset_utils import alternativeSeperation
from model_utils import ModelEngine

Before beggining any machine learning task, one have to prepare project specific data. Tiny Imagenet dataset comes with 3 folders; for training, for validation and for testing. Training folder is well structured for PyTorch's prebuilt ImageFolder dataset handler. However, validation folder has all images in one folder, and labels of them in a text file. I implemented 2 alternatives for dealing such a task. 
- (Alternative) First one of the alternatives, is an extension for PyTorch's "nn.Dataset" class, gets images according to text file. This alternative introduces some overhead to system since it labels all images as the training goes. Further explaination can be obtained from class doc.
- (Preferred) Second alternative is a function that seperates images among class folders that created correspondingly. With this function, the need of RAM usage for holding label information becomes unnecessary. This function also enables user to use PyTorch's "ImageFolder" implementation.

In [2]:
# alternativeSeperation()

Definition: A loss landscape is a multi-dimensional representation of loss functions. For more information please take a look at [1].

In computer vision tasks, most of the loss landscapes are not convex. This property of loss landscapes makes it hard to find the global optima. There are 3 main elements that affects the loss landscape severely; loss function, data distribution and model architecture. (Other hyper-parameters also affect loss landscapes) In computer vision, this data distribution can be adjusted slightly to lead the machine learning pipeline to find the global optima. (or at least a good local optima)

- First adjustment should be channel normalization. Normalization is an important step to have an easier loss landscape. In machine learning projects, all features may have their own distributions. However, numerically, a distribution significantly different from other ones (e.g. a distribution with much larger samples) may affect loss landscape severely and decreases gradients on a dimension while increases on another one. (In other words, the optimizer may think that a feature is more important than other one) With channel normalization, one may prevent that issue. This process can be done by statistically analyzing datasets channel distribution. 

- Data augmentation may be nice to have tool for exploration of loss landscape. A dataset can only represent a partition of a real distribution. With data augmentation, one can slightly increase the coverage of that representation and in some ways enhance it. Some of the benefits of data augmentation are:
    - May prevent overfitting (in some cases) for classes by changing translation, rotation, angle etc.
    - May balance a dataset that is unbalanced so, the bias of model can be decreased.
    - Increase of coverage may lead to better generalization.
    
    <br>Some ways to augment image data are:</br>
   
   
    - Random cropping-resizing-rotation
    - Horizontal and vertical flips
    - Random erasure
    - Hue and color shifts
    - Shearing, tilting and skewing
    - Adding jitter and random noise
    - Random distortions
    
However, data augmentation is just replicating an image with some distortions and noise. Overdoing may harm generalization. In this project, very little amount of augmentation will be used.

In this part, we will find out which augmentation performs better for our case.

[1] https://arxiv.org/pdf/1712.09913.pdf

In [80]:
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                 std=[0.5, 0.5, 0.5])
transformations_simple = {
    'train': transforms.Compose([
        transforms.CenterCrop(57),
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(57),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(57),
        transforms.ToTensor(),
        normalize
    ])
}

data_dir = "data"

image_datasets_simple = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transformations_simple[x])
                  for x in ['train', 'val']}
dataloaders_simple = {x: DataLoader(image_datasets_simple[x], batch_size=64,
                             shuffle=True)
               for x in ['train', 'val']}

In [81]:
transformations = {
    'train': transforms.Compose([
        transforms.RandomCrop(57),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(57),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(57),
        transforms.ToTensor(),
        normalize
    ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transformations[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=64,
                             shuffle=True)
               for x in ['train', 'val']}

One may need to create a network architecture to experiment on. As mentioned before, in this notebook, a single simple network will be created to evaluate some hyper-parameters. Due to limited hardware, optimization for some hyper-parameters will be done with this network. This approach is not accurate, as different hyper-parameters may perform better or worse under different architectures. However, this method may, at least, provide a good starting point for further optimization, as this procedure gives a rough idea on how augmentations affect the loss landscape.

In [5]:
class ConvNetExperimental(nn.Module):
    """
    Initial model to experiment on.
    """
    def __init__(self):
        super(ConvNetExperimental, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1),
            nn.ReLU(),  
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  
            nn.Conv2d(in_channels=512, out_channels=384, kernel_size=3, stride=1),
            nn.ReLU(),
        )
        self.fc = nn.Linear(384 * 3 * 3, 200)
        
    def forward(self, inputs):
        x = self.feature_extractor(inputs)
        return self.fc(x.view(x.size(0), -1))
    
# model = ConvNetExperimental()
# out = model(torch.zeros(1, 3, 57, 57))
# out.shape

In [6]:
simple_model = ConvNetExperimental()
simple_criterion = nn.CrossEntropyLoss()
simple_optimizer = optim.Adam(simple_model.parameters())
simple_engine = ModelEngine(simple_model, simple_criterion, simple_optimizer, model_name="SimpleModel1")

Using device cuda:0


In [7]:
simple_engine.fit(dataloaders_simple)

The created network is rather simple, however, is able to generalize the given dataset relatively. The simplicity of network also makes the experimenting procedure faster. Please note that, the given dataset is just a center cropped version of the original dataset to 57x57 resolution. One can observe the training loss and validation loss curves from TensorBoard. From these curves, one can observe that the model has a slight overfit issue. This slight overfit may indicate a bad local optima. It is a good practice to augment the data and find the optimal augmentations for the current network architectures. As mentioned before, this may lead to a better loss landscape, thus better generalization.

In [25]:
augmented_model = ConvNetExperimental()
augmented_criterion = nn.CrossEntropyLoss()
augmented_optimizer = optim.Adam(augmented_model.parameters())
augmented_engine = ModelEngine(augmented_model, augmented_criterion, augmented_optimizer, model_name="AugmentedModel1")

Using device cuda:0


In [26]:
augmented_engine.fit(dataloaders)

Graphics named "AugmentedModel1" shows the learning curves for a dataset augmentation with "randomly cropped images". As it can be seen from TensorBoard graphics, despite of the worse training time loss values, augmentation lead to better generalization for both practices. Validation curve shows that on each epoch, randomizing may lead better or worse training sequences.

Most of the regularization methods mainly aim for better generalization, despite their different approaches. However, one can say that regularization methods are mostly to "restrict the power of the method and prevent overfitting" on irreducible error for specific model-dataset pair. 

The regularization parameters are omitted for those experiments, since they are not much needed. This is because the models are not powerful enough and thus, restricting power is not the highest priority. The gap between human level is much higher than the validation and training set. 

Important note: Powerful models will not be investigated in this notebook as they are expensive to train and hardware for such expensive experiments is not present.

In [82]:
class ConvNetWithResidualExperimental(nn.Module):
    """
    Enhanced version of the previously created ConvNet with residual connection.
    """
    def __init__(self, drop_rate = 0):
        """
        Initialization function
        :param drop_rate: Drop rate for dropout layer
        """
        super(ConvNetWithResidualExperimental, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)
        self.max_pool = nn.MaxPool2d(2)
        self.avg_pool = nn.AvgPool2d(12)
        self.relu = nn.ReLU()
        self.size_fix1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.size_fix2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(drop_rate)

        self.fc = nn.Linear(1024, 200)
        
    def forward(self, inputs):
        residual1 = inputs
        x = self.conv1(inputs)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = torch.cat((x, self.size_fix1(residual1)), 1)
        x = self.max_pool(x)
        residual2 = x
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = self.relu(x)
        x = torch.cat((x, self.size_fix2(residual2)), 1)
        x = self.max_pool(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)
    
# model = ConvNetWithResidualExperimental()
# out = model(torch.zeros(1, 3, 57, 57))
# out.shape

In [83]:
resnet_model = ConvNetWithResidualExperimental()
resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_model.parameters())
resnet_engine = ModelEngine(resnet_model, resnet_criterion, resnet_optimizer, model_name="ResNetModel1")

Using device cuda:0


In [84]:
resnet_engine.fit(dataloaders)

The model seems to overfit on the dataset. Some measures of regularization can be adapted to solve this issue. Some of the regularization methods did not applied till now are:
- L1-L2 regularization
- Dropout
- (Further) Augmentation
- Stochastic Pooling [1]

Further augmentation and stochastic pooling will not be implemented for this project due to time restrictions.

L2 regularization is a widely adapted regularization method for prevent underfitting and better generalization. The simple mathematical formula of the L2 regularization is shown below.

$$Loss(\hat{y}, y) + \lambda * \sum_j {\beta_j^2}$$

As it can be seen from the formula, by adding a penalty to each parameter( $\beta_j$ ) for being large, this regularizer forces parameters to be as low as possible. With that way, the search space of parameters is being forced to shrink. This is also called "weight decay". 

However, L2 regularization shouldn't be applied with the classic Adam optimizer. Please take a look at second item of the "Notes On Optimizers" section. 

## Notes On Optimizers
- While adaptive methods (especially Adam optimizer) are widely used for experimenting and hyper-parameter tuning, a sequence of partial training procedures employing SGD optimizer by watching learning curve carefully and adapting learning rate accordingly among these training procedures performs the best for the final training. Please note that this performance may vary depending on Machine Learning Engineer's insights.

- Adam optimizer is very popular as it is good for experimenting and performs on acceptable levels. While Adam performs good on practice, theoretically, Adam optimizer has a calculation error in adaptive weight decay section. However, this issue only affects  the earlier stages of the training procedure and does not impact the procedure heavily.

In [47]:
resnet_model = ConvNetWithResidualExperimental(.2)
resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_model.parameters(), weight_decay=.00001)
resnet_engine = ModelEngine(resnet_model, resnet_criterion, resnet_optimizer, model_name="ResNetModel3")

Using device cuda:0


In [48]:
resnet_engine.fit(dataloaders)

There are some more hyper-parameters to tune after deciding on network architecture. These hyper-parameters are also to be tuned. These hyper-parameters are:
- Loss function: A loss function has a crucial effect on loss landscape, and should be selected accordingly to the given task. Cross Entropy Loss is good with multi-class classification tasks. Logarithmic property of the function improves "convexity" of the loss function. Further information can be found at PyTorch's doc[1].
- Optimizer: While the standard optimizer for any given loss function is stochastic gradient descent (SGD), an adaptive optimizer performs better on hyper-parameter tuning task. Adam optimizer is considered as a good adaptive optimizer choice for various tasks. Please take a look at "notes on optimizers" part for further understandings on how optimizers can be used.
- Scheduler: A scheduler adapts learning rate of the optimizer accordingly. While adaptive optimizers mostly do not need a scheduler, a SGD optimizer with a good scheduler may outperform all other optimizers.

The training procedure will be handled by "Model Engine" written for the given projects. This class eases the training, validation and prediction process. One can look at documentation of "ModelEngine" class for further information.

[1] https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss

In [85]:
import numpy as np
import PIL
import cv2


class OtsuThreshold(object):
    """
    Otsu thresholding method for transforming input image with a masked image.
    Please note that commented out partition is the numpy implementation. Since it's too slow,
    OpenCV version is being employed.
    """
    def __init__(self):
        pass

    def __call__(self, sample):
        sample = np.array(sample)
        gray = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, mask = cv2.threshold(blur, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#         gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

#         pixel_number = gray.shape[0] * gray.shape[1]
#         mean_weigth = 1.0 / pixel_number
#         his, bins = np.histogram(gray, np.arange(0, 257))
#         final_thresh = -1
#         final_value = -1
#         intensity_arr = np.arange(256)
#         for t in bins[1:-1]:
#             pcb = np.sum(his[:t])
#             pcf = np.sum(his[t:])
#             Wb = pcb * mean_weigth
#             Wf = pcf * mean_weigth

#             mub = np.sum(intensity_arr[:t] * his[:t]) / float(pcb)
#             muf = np.sum(intensity_arr[t:] * his[t:]) / float(pcf)
#             value = Wb * Wf * (mub - muf) ** 2

#             if value > final_value:
#                 final_thresh = t
#                 final_value = value
#         mask = gray.copy()
#         mask[gray > final_thresh] = 1
#         mask[gray < final_thresh] = 0
        return PIL.Image.fromarray(np.uint8(sample * mask[..., None]))


In [86]:
transformations = {
    'train': transforms.Compose([
        transforms.Resize(57),
        OtsuThreshold(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(57),
        OtsuThreshold(),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(57),
        OtsuThreshold(),
        transforms.ToTensor(),
        normalize
    ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transformations[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=64,
                             shuffle=True)
               for x in ['train', 'val']}

In [87]:
class ConvNetWithResidualExperimental(nn.Module):
    def __init__(self, drop_rate = 0):
        super(ConvNetWithResidualExperimental, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)
        self.max_pool = nn.MaxPool2d(2)
        self.avg_pool = nn.AvgPool2d(12)
        self.relu = nn.ReLU()
        self.size_fix1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.size_fix2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(drop_rate)

        self.fc = nn.Linear(1024, 200)
        
    def forward(self, inputs):
        residual1 = inputs
        x = self.conv1(inputs)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = torch.cat((x, self.size_fix1(residual1)), 1)
        x = self.max_pool(x)
        residual2 = x
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = self.relu(x)
        x = torch.cat((x, self.size_fix2(residual2)), 1)
        x = self.max_pool(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)
    
# model = ConvNetWithResidualExperimental()
# out = model(torch.zeros(1, 3, 57, 57))
# out.shape

In [89]:
resnet_model = ConvNetWithResidualExperimental()
resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_model.parameters())
otsu_engine = ModelEngine(resnet_model, resnet_criterion, resnet_optimizer, model_name="ResNetModelOtsu1")

Using device cuda:0


In [90]:
otsu_engine.fit(dataloaders)